In [1]:
import os
import numpy as np
from time import time
import datetime
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd

In [2]:
filepath ='/home/dell/Xinda/SVM/server/Audio/data_opensmile/eGeMAPs_Arousal.csv'
data = pd.read_csv(filepath)
x = data.iloc[:, :-1]  # 数据特征
y = data.iloc[:,-1]  # 标签

# 将数据划分为训练集和测试集，test_size=.3表示30%的测试集, 随机数种子, 保证可复现性
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=420)
x_train

,audio_1,audio_2,audio_3,audio_4,audio_5,audio_6,audio_7,audio_8,audio_9,audio_10,...,audio_79,audio_80,audio_81,audio_82,audio_83,audio_84,audio_85,audio_86,audio_87,audio_88
4097,20.75965,0.103017,18.98845,20.77354,23.03446,4.046011,75.55734,44.75021,47.79361,38.03314,...,0.014480,-0.007094,0.054649,5.000000,1.492537,0.550000,0.350000,0.100000,0.000000,-45.19889
13130,23.25654,0.097709,20.99730,23.60110,24.91517,3.917873,86.96716,131.52920,33.32082,32.29058,...,0.000435,-0.012817,0.032778,4.810997,3.169014,0.216667,0.136382,0.090000,0.102835,-47.95344
12464,26.15638,0.163852,23.70398,25.94531,29.55886,5.854879,206.31090,209.02770,66.84982,74.16927,...,0.013834,-0.009255,0.036738,2.578797,1.466276,0.376000,0.204900,0.236667,0.318311,-39.77215
14322,31.91281,0.284221,24.28072,36.09782,39.02469,14.743970,444.99770,612.65390,132.84680,171.36600,...,0.035442,-0.009925,0.117786,4.987532,1.269036,0.730000,0.682114,0.050000,0.025495,-40.40957
1330,31.70204,0.132445,28.05466,33.43954,34.52127,6.466608,121.94640,63.69006,83.90160,109.43880,...,0.006656,-0.016533,0.032976,4.469274,2.312139,0.390000,0.227706,0.045000,0.025000,-43.86897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13087,29.28543,0.184819,27.09436,31.50656,32.86273,5.768375,643.01290,783.54660,28.61181,15.67835,...,0.029166,-0.022895,0.044269,5.027933,1.734104,0.510000,0.314960,0.100000,0.000000,-46.18982
4671,25.12424,0.052365,23.88896,25.40715,26.27013,2.381168,127.42260,107.60610,48.79199,83.70647,...,-0.005003,-0.011607,0.026468,3.243243,2.197802,0.261250,0.133645,0.153333,0.113529,-50.11962
4486,38.20967,0.076671,37.27678,38.68452,39.72790,2.451126,30.51731,32.05616,672.08260,631.62220,...,-0.024289,-0.004192,0.024753,0.633914,0.638978,0.182500,0.165133,1.088000,2.001943,-50.94144
13384,37.87624,0.081298,36.33438,39.00095,39.86536,3.530983,155.33140,182.30790,22.36949,28.94270,...,0.029331,-0.017906,0.040637,4.285714,3.030303,0.200000,0.127279,0.110000,0.115542,-48.39392


In [3]:
# 修正测试集和训练集的索引
for i in [x_train, x_test, y_train, y_test ]:
    i.index  = range(i.shape[0])

# 标准化
scaler_x = StandardScaler()
x_train_std = scaler_x.fit_transform(x_train)
x_test_std = scaler_x.fit_transform(x_test)
print(x_train_std.shape)
print(y_train.shape)

(10244, 88)
(10244,)


In [4]:
def getPvar(vals, mean):
    N = len(vals)
    su = 0
    for i in range(len(vals)):
        su = su + ((vals[i]-mean)*(vals[i]-mean))
    pvar = (1/N) * su
    return pvar

def getMean(vals):
    su = 0
    for i in range(len(vals)):
        su = su + vals[i]
    mean = su/(len(vals))
    return mean

def getMeanofDiffs(xvals, yvals):
    su = 0
    for i in range(len(xvals)):
        su = su + ((xvals[i] - yvals[i])*(xvals[i] - yvals[i]))
    meanodiffs = su/(len(xvals))
    return meanodiffs

def getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp):
    bottom = pvarfe + pvarexp + ((meanfe - meanexp)*(meanfe - meanexp))
    answer = 1 - (meanofdiff / bottom)
    return answer

In [5]:
RMSE_list = []
Spearman_list = []
CCC_list = []
clf = SVR(kernel = 'rbf', gamma = 0.028933584758977834, cache_size=5000)
clf.fit(x_train_std, y_train)
result = clf.predict(x_test_std)
# accuracy
rmse = sqrt(mean_squared_error(y_test, result))
print("(1) Evaluation - RMSE = ", rmse)

# Spearman
data = {'result':result, 'y_test':y_test}
df = pd.DataFrame(data, columns=['result','y_test'])
spearman = df.corr(method="spearman" )
print("(2) Evaluation - Spearmman = \n", spearman)

# CCC
prediction = result
ground = y_test
meanfe = getMean(ground)
meanexp = getMean(prediction)
meanofdiff = getMeanofDiffs(ground,prediction)
pvarfe = getPvar(ground, meanfe)
pvarexp = getPvar(prediction, meanexp)
ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
print('(3) Evaluation - CCC =  ' + str(ccc))

df = pd.DataFrame(data={"opensmile_prediction_a": prediction, "opensmile_groundtruth_a": y_test.values.tolist()})
df.to_csv("eval_opensmile_arousal.csv")
print("save success!")

(1) Evaluation - RMSE =  0.3251203526367954
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.393887
y_test  0.393887  1.000000
(3) Evaluation - CCC =  0.398916519812354
save success!


### Gamma

In [5]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

gamma_range = np.logspace(-10, 1, 10, base=2) # 返回13个数字，底是2
print("gamma_rang:", gamma_range)

for gamma_item in gamma_range:
    count=1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count+1
    clf = SVR(kernel = 'rbf', gamma = gamma_item, cache_size=5000)
    clf.fit(x_train_std, y_train)
    
    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)
    
    # Spearman
    data = {'result':result, 'y_test':y_test}
    df = pd.DataFrame(data, columns=['result','y_test'])
    spearman = df.corr(method="spearman" )
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)
    
    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground,prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print("Gamma = ", gamma_range[CCC_list.index(max(CCC_list))])
print("RMSE = ", RMSE_list[CCC_list.index(max(CCC_list))])
print("Spearman = ", Spearman_list[CCC_list.index(max(CCC_list))])
print("CCC = ", max(CCC_list) ) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

gamma_rang: [9.76562500e-04 2.27837703e-03 5.31558594e-03 1.24015707e-02
 2.89335848e-02 6.75037337e-02 1.57490131e-01 3.67433623e-01
 8.57243983e-01 2.00000000e+00]
Start-1, gamma=0.0009765625
(1) Evaluation - RMSE =  0.3579885587978931
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.266694
y_test  0.266694  1.000000
(3) Evaluation - CCC =  0.09825956346173814
00:32:720254


Start-1, gamma=0.0022783770304221013
(1) Evaluation - RMSE =  0.3515280343851432
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.311262
y_test  0.311262  1.000000
(3) Evaluation - CCC =  0.14739699640743342
00:36:055216


Start-1, gamma=0.005315585938181161
(1) Evaluation - RMSE =  0.3422411204606352
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.353779
y_test  0.353779  1.000000
(3) Evaluation - CCC =  0.22544048122201443
00:43:700266


Start-1, gamma=0.012401570718501561
(1) Evaluation - RMSE =  0.33115462633679765
(2) Eva

### Gamma-2

In [ ]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

gamma_range = np.linspace(0.012,0.06,10)
print("gamma_rang:", gamma_range)

for gamma_item in gamma_range:
    count=1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count+1
    clf = SVR(kernel = 'rbf', gamma = gamma_item, cache_size=5000)
    clf.fit(x_train_std, y_train)
    
    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)
    
    # Spearman
    data = {'result':result, 'y_test':y_test}
    df = pd.DataFrame(data, columns=['result','y_test'])
    spearman = df.corr(method="spearman" )
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)
    
    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground,prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print("Gamma = ", gamma_range[CCC_list.index(max(CCC_list))])
print("RMSE = ", RMSE_list[CCC_list.index(max(CCC_list))])
print("Spearman = ", Spearman_list[CCC_list.index(max(CCC_list))])
print("CCC = ", max(CCC_list) ) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))
print("Test over")